# Introdução e limpeza

O scraping foi realizado com a biblioteca selenium e está otimizado para que seja replicável em outras máquinas e computadores.

Cada .csv possui as seguintes colunas:

- `Unnamed: 0`: sujeira do processo de scrapping
- `Rank`: Indica o lugar no ranking que a música ficou, entre 1 a 100, naquela semana que ela entrou no chart.
- `Previous Rank`: Indica o lugar no raking que a música ficou na semana anterior. Null e similares ocorre quando a música não esteve no ranking.
- `Track Name`: Nome da faixa 
- `Artist Names`: Nome do artista
- `Weeks on Chart`: Quantas semanas a faixa ficou nos charts das 100 mais reproduzidas no Youtube Brasil
- `Views`: Número de visualizações do vídeos naquela semana
- `Weekly Growth`: Crescimento percentual de visualizações em relação a semana anterior
- `Youtube URL`: Url de acesso ao vídeo
- `week_ref`: coluna introduzida no processo de scrapping para facilitar manipulações. Varia de 0 a 70. Quanto maior o número, maior a distância do dia de realização do scrapping, sendo 0 a última semana disponível de dados.
- `week_open`: Data de início da semana referente ao chart
- `week_close`: Data de fechamento da semana referente ao chart 

Mais a frente, será adicionada a coluna
- `frontline`: valores booleanos 0 e 1. A coluna informará se a faixa faz parte da categoria "frontline", grupo de faixas que foram lançadas a menos de 42 semanas.

Neste notebook, concatenarei os dataframes gerados a partir do script de scrapping, permitindo que prossigamos com a análise. Também será ralizado tratamento do dataframe e `casting` para melhor adequar os tipos de dados. O markdown será mantido em português e o código utilizará variáveis e comentários em inglês para manter o padrão das convenções do `python`

In [41]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

path = '../data/raw'
files = os.listdir(path)

csv_files = [f for f in files if f.endswith('.csv')]

df_list = []

for csv_file in csv_files:
    df = pd.read_csv(os.path.join(path, csv_file))
    df_list.append(df)

# Concatenaing all .csvs into a one big file 
weekly_charts = pd.concat(df_list)
weekly_charts.reset_index(drop=True, inplace=True)

Checando o dataframe criado:

In [42]:
weekly_charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7100 entries, 0 to 7099
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      7100 non-null   int64  
 1   Rank            7100 non-null   int64  
 2   Previous Rank   6000 non-null   float64
 3   Track Name      7100 non-null   object 
 4   Artist Names    7100 non-null   object 
 5   Weeks on Chart  7100 non-null   int64  
 6   Views           7100 non-null   int64  
 7   Weekly Growth   6000 non-null   object 
 8   YouTube URL     7100 non-null   object 
 9   week_ref        7100 non-null   int64  
 10  week_open       7100 non-null   object 
 11  week_close      7100 non-null   object 
dtypes: float64(1), int64(5), object(6)
memory usage: 665.8+ KB


Removendo a coluna `Unnamed: 0`, sujeira do processo de scrap.

In [43]:
weekly_charts = weekly_charts.drop(["Unnamed: 0"], axis=1)
weekly_charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7100 entries, 0 to 7099
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Rank            7100 non-null   int64  
 1   Previous Rank   6000 non-null   float64
 2   Track Name      7100 non-null   object 
 3   Artist Names    7100 non-null   object 
 4   Weeks on Chart  7100 non-null   int64  
 5   Views           7100 non-null   int64  
 6   Weekly Growth   6000 non-null   object 
 7   YouTube URL     7100 non-null   object 
 8   week_ref        7100 non-null   int64  
 9   week_open       7100 non-null   object 
 10  week_close      7100 non-null   object 
dtypes: float64(1), int64(4), object(6)
memory usage: 610.3+ KB


Ajustando tipos de colunas para datetime:

In [44]:
# Convertting columns to datetime.
weekly_charts['week_open'] = pd.to_datetime(weekly_charts["week_open"])
weekly_charts['week_close'] = pd.to_datetime(weekly_charts["week_close"])
weekly_charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7100 entries, 0 to 7099
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Rank            7100 non-null   int64         
 1   Previous Rank   6000 non-null   float64       
 2   Track Name      7100 non-null   object        
 3   Artist Names    7100 non-null   object        
 4   Weeks on Chart  7100 non-null   int64         
 5   Views           7100 non-null   int64         
 6   Weekly Growth   6000 non-null   object        
 7   YouTube URL     7100 non-null   object        
 8   week_ref        7100 non-null   int64         
 9   week_open       7100 non-null   datetime64[ns]
 10  week_close      7100 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(4), object(4)
memory usage: 610.3+ KB


Verificando e tratando `nulls`, `NaNs` e `<NA>`

In [45]:
weekly_charts.isna().any()

Rank              False
Previous Rank      True
Track Name        False
Artist Names      False
Weeks on Chart    False
Views             False
Weekly Growth      True
YouTube URL       False
week_ref          False
week_open         False
week_close        False
dtype: bool

In [46]:
weekly_charts.isnull().any()

Rank              False
Previous Rank      True
Track Name        False
Artist Names      False
Weeks on Chart    False
Views             False
Weekly Growth      True
YouTube URL       False
week_ref          False
week_open         False
week_close        False
dtype: bool

`Previous Rank` apresenta o tipo `float64` e `Weekly Growth` aparece como objeto (`string`). Ambas colunas também possuem objetos `NaNs` e `<NA>`. Cerca de 1100 - o que é justificável, visto que algumas músicas entram nos _charts_ sem necessariamente ter estado anteriormente. Portanto, não há, nesses casos, nem rank prévio, nem crescimento semanal. Entretanto, considerando que os rankings variam de 1 a 100, o ideal é que a `series` de `Previous Rank` se apresente como integers, em razão da precisão. O mesmo vale para `Weekly Growth`.

Como os métodos mais comuns de lidar com nulos não fazem sentido nesse caso, visto que o `drop` de uma linha pode ser prejudicial à nossa análise e a substituição dos `NaNs` e `<NA>` por medianas tampouco faz sentido, substituiremos nulos por `zeros` e optaremos por converter o tipo float64 para int64, tipo específico que é capaz de receber nulos e `NaNs`.

In [47]:
#Convertting NaNs and <NA> and nulls to Zero
weekly_charts["Previous Rank"].fillna(0, inplace=True)
#Convertting to Int64
weekly_charts["Previous Rank"] = weekly_charts['Previous Rank'].astype('int64')


In [48]:
weekly_charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7100 entries, 0 to 7099
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Rank            7100 non-null   int64         
 1   Previous Rank   7100 non-null   int64         
 2   Track Name      7100 non-null   object        
 3   Artist Names    7100 non-null   object        
 4   Weeks on Chart  7100 non-null   int64         
 5   Views           7100 non-null   int64         
 6   Weekly Growth   6000 non-null   object        
 7   YouTube URL     7100 non-null   object        
 8   week_ref        7100 non-null   int64         
 9   week_open       7100 non-null   datetime64[ns]
 10  week_close      7100 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(5), object(4)
memory usage: 610.3+ KB


In [49]:
weekly_charts.head()

,Rank,Previous Rank,Track Name,Artist Names,Weeks on Chart,Views,Weekly Growth,YouTube URL,week_ref,week_open,week_close
0,1,1,Malvada,Zé Felipe,2,21185310,9.8%,https://www.youtube.com/watch?v=r0mNwyywHIY,70,2022-02-04,2022-02-10
1,2,2,Malvadão 3,Xamã,8,17063733,-3.7%,https://www.youtube.com/watch?v=i9NOkjmBszo,70,2022-02-04,2022-02-10
2,3,3,Depende,Dj Guuga,8,11911917,8.7%,https://www.youtube.com/watch?v=v9geHIUOxeA,70,2022-02-04,2022-02-10
3,4,4,Vontade De Morder,Simone & Simaria & Zé Felipe,4,11243771,3.4%,https://www.youtube.com/watch?v=bgf18kANm_k,70,2022-02-04,2022-02-10
4,5,6,Mal Feito (Ao Vivo),Hugo & Guilherme & Marília Mendonça,4,9643194,-5.6%,https://www.youtube.com/watch?v=kyfXEMqvLgU,70,2022-02-04,2022-02-10


A coluna `Weekly Growth`, por sua vez, é incapaz de ser convertida para valor numérico em razão do elemento "%". Para isso, converteremos porcentagens em decimais

In [50]:
weekly_charts['Weekly Growth'] = weekly_charts['Weekly Growth'].str.rstrip('%').astype('float') / 100.0
weekly_charts["Weekly Growth"].fillna(0, inplace=True)
weekly_charts.info()
weekly_charts.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7100 entries, 0 to 7099
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Rank            7100 non-null   int64         
 1   Previous Rank   7100 non-null   int64         
 2   Track Name      7100 non-null   object        
 3   Artist Names    7100 non-null   object        
 4   Weeks on Chart  7100 non-null   int64         
 5   Views           7100 non-null   int64         
 6   Weekly Growth   7100 non-null   float64       
 7   YouTube URL     7100 non-null   object        
 8   week_ref        7100 non-null   int64         
 9   week_open       7100 non-null   datetime64[ns]
 10  week_close      7100 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(5), object(3)
memory usage: 610.3+ KB


,Rank,Previous Rank,Track Name,Artist Names,Weeks on Chart,Views,Weekly Growth,YouTube URL,week_ref,week_open,week_close
0,1,1,Malvada,Zé Felipe,2,21185310,0.098,https://www.youtube.com/watch?v=r0mNwyywHIY,70,2022-02-04,2022-02-10
1,2,2,Malvadão 3,Xamã,8,17063733,-0.037,https://www.youtube.com/watch?v=i9NOkjmBszo,70,2022-02-04,2022-02-10
2,3,3,Depende,Dj Guuga,8,11911917,0.087,https://www.youtube.com/watch?v=v9geHIUOxeA,70,2022-02-04,2022-02-10
3,4,4,Vontade De Morder,Simone & Simaria & Zé Felipe,4,11243771,0.034,https://www.youtube.com/watch?v=bgf18kANm_k,70,2022-02-04,2022-02-10
4,5,6,Mal Feito (Ao Vivo),Hugo & Guilherme & Marília Mendonça,4,9643194,-0.056,https://www.youtube.com/watch?v=kyfXEMqvLgU,70,2022-02-04,2022-02-10


Salvaremos o dataframe resultante em `.csv` como referência e `pickle` para manter os tipos de dados, facilitando a interação dos arquivos entre os notebooks.

In [51]:
weekly_charts.to_csv("../data/trusted/weekly_charts.csv", index=False)
weekly_charts.to_pickle("../data/trusted/weekly_charts.pkl")
